# A simple example of RNN
In this problem, we want to predict the next steps of a walking path on a plane using __RNN__. 
## Generating data

In [64]:
import numpy as np
import random

size = 100000
seq_len = 20
batch_size = 128
state_size = 100

In [2]:
# randomly generate a walking (on a plance) path, each point has two coordinates
# stack sequences of length seq_len into Xs and Ys, where Ys is one step forward from Xs.
def gen_data(size = 10000, seq_len = 20):
    path = np.array(np.random.normal(0.5,.01,size=(size + seq_len,2)))
    Xs = np.zeros((size,seq_len,2))
    Ys = np.zeros((size,seq_len,2))
    for i in xrange(size):
        Xs[i,:,:] = path[i : i+seq_len , :]
        Ys[i,:,:] = path[i+1 : i+1+seq_len , :]
    return (Xs, Ys, path)

In [3]:
(Xs, Ys, path) = gen_data(size, seq_len)
Xs.shape, Ys.shape, path.shape

((100000, 20, 2), (100000, 20, 2), (100020, 2))

## 1. Using Keras
### Building the model

In [4]:
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.


In [5]:
print('Build model...')
model = Sequential()
model.add(LSTM(state_size, return_sequences=True, input_shape=(seq_len, 2)))  
model.add(LSTM(state_size, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(2)))
model.compile(loss='mse', optimizer='rmsprop')
print ('model is made')

Build model...
model is made


In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 128)           67072     
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 2)             258       
Total params: 198,914
Trainable params: 198,914
Non-trainable params: 0
_________________________________________________________________
None


### Print a sample before training

In [10]:
def print_array(a):
    # A helper function for printing an array
    # The shape of the array is 1 x None x 2, print with 3 decimal places and with groups of 2 numbers
    a = np.around(a,3)
    for item in a[0,:,:]:
        print item,
    
def print_sample():
    # A helper function for printing a sample with predicted future steps
    start_index = random.randint(0,size)
    seq_len = 20
    print('--------------- Generating with seed: ----------------')
    x = np.zeros((1,seq_len,2))
    x[0,:,:] = path[start_index:start_index+seq_len]
    print_array(x)
    print()
    print('--------------------- Prediting ----------------------')
    
    for i in range(20):
        preds = model.predict(x, verbose=0)
        next_step = np.zeros((1, 1, 2))
        next_step[0,:,:] = preds[:,-1,:]
        print_array(next_step)
        past_steps = x[:,1:,:]
        x = np.concatenate(( past_steps, next_step ), axis = 1)
        
    print()
        

In [11]:
print("---------- Before Training ----------")
print_sample()

---------- Before Training ----------
--------------- Generating with seed: ----------------
[ 0.498  0.499] [ 0.488  0.501] [ 0.511  0.481] [ 0.503  0.507] [ 0.511  0.501] [ 0.492  0.494] [ 0.497  0.498] [ 0.504  0.488] [ 0.509  0.507] [ 0.48   0.515] [ 0.499  0.5  ] [ 0.508  0.505] [ 0.511  0.48 ] [ 0.51  0.5 ] [ 0.491  0.492] [ 0.517  0.495] [ 0.499  0.471] [ 0.498  0.517] [ 0.511  0.481] [ 0.51   0.499] ()
--------------------- Prediting ----------------------
[-0.027  0.024] [-0.027  0.023] [-0.027  0.022] [-0.026  0.021] [-0.024  0.019] [-0.021  0.018] [-0.018  0.016] [-0.016  0.014] [-0.013  0.012] [-0.011  0.01 ] [-0.009  0.008] [-0.007  0.006] [-0.005  0.005] [-0.004  0.003] [-0.003  0.002] [-0.002  0.001] [-0.002  0.001] [-0.001 -0.   ] [-0.001 -0.   ] [-0.    -0.001] ()


### Training the rnn with demonstration at each iteration

In [12]:
for iteration in range(1,3):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    print_sample()
    history = model.fit(Xs, Ys, batch_size = batch_size, epochs = 1, verbose = 1)
    if iteration==1 or iteration==3 or iteration==5 or iteration==10 or iteration==20 or iteration==30 or iteration==40 or iteration==50:
        model.save_weights('rnn_rw_' + str(iteration) + '.h5', overwrite=True)
    print('loss is')
    print(history.history['loss'][0])
    print(history)
    print()
    model.save('rnn_rw')

()
--------------------------------------------------
('Iteration', 1)
--------------- Generating with seed: ----------------
[ 0.491  0.506] [ 0.479  0.481] [ 0.501  0.504] [ 0.495  0.498] [ 0.504  0.497] [ 0.506  0.501] [ 0.497  0.478] [ 0.49   0.509] [ 0.507  0.493] [ 0.508  0.498] [ 0.497  0.506] [ 0.517  0.509] [ 0.499  0.513] [ 0.502  0.497] [ 0.525  0.494] [ 0.509  0.483] [ 0.497  0.485] [ 0.496  0.515] [ 0.516  0.495] [ 0.501  0.505] ()
--------------------- Prediting ----------------------
[-0.027  0.024] [-0.027  0.023] [-0.027  0.022] [-0.026  0.021] [-0.024  0.019] [-0.021  0.018] [-0.019  0.016] [-0.016  0.014] [-0.013  0.012] [-0.011  0.01 ] [-0.009  0.008] [-0.007  0.006] [-0.005  0.005] [-0.004  0.003] [-0.003  0.002] [-0.002  0.001] [-0.002  0.   ] [-0.001 -0.   ] [-0.001 -0.001] [-0.    -0.001] ()
Epoch 1/1
100000/100000 [==============================] - 123s 1ms/step - loss: 0.0034
loss is
0.00338432063181
()
()
--------------------------------------------------
('I

In [13]:
print_sample()

--------------- Generating with seed: ----------------
[ 0.511  0.489] [ 0.495  0.488] [ 0.505  0.501] [ 0.497  0.487] [ 0.499  0.504] [ 0.496  0.507] [ 0.517  0.5  ] [ 0.496  0.517] [ 0.489  0.509] [ 0.506  0.508] [ 0.503  0.49 ] [ 0.503  0.502] [ 0.485  0.52 ] [ 0.507  0.497] [ 0.497  0.507] [ 0.509  0.499] [ 0.504  0.514] [ 0.492  0.497] [ 0.496  0.508] [ 0.504  0.505] ()
--------------------- Prediting ----------------------
[ 0.49   0.489] [ 0.489  0.488] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] [ 0.488  0.487] ()


## 2. Using TensorFlow API

In [65]:
def get_batch(Xs, Ys, batch_size = 128):
    s = int(np.random.uniform(0, len(Xs) - batch_size))
    data_x = Xs[s : s+batch_size]
    data_y = Ys[s : s+batch_size]
    return data_x, data_y

In [66]:
data_x, data_y = get_batch(Xs, Ys)
data_x.shape, data_y.shape

((128, 20, 2), (128, 20, 2))

In [78]:
import tensorflow as tf
learning_rate = 0.01
num_layers = 3

### Definition of the LSTM cells

In [94]:
tf.reset_default_graph()
tf.set_random_seed(1)

# Placeholders
x = tf.placeholder(tf.float32, [None, seq_len, 2], name = 'data_x')
y = tf.placeholder(tf.float32, [None, seq_len, 2], name = 'data_y')
dropout = tf.placeholder(tf.float32)

print('building the model')

# create 2 LSTM cells with dropout between layers
rnn_layers = []
for _ in range(num_layers):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = 1.0 - dropout)
    rnn_layers.append(cell)
    
# create a RNN cell composed sequentially of a number of RNNCells
multi_rnn_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)

# outputs is a tensor of shape [batch_size, seq_len, state_size]
# state is a N-tuple where N is the number of LSTMCells containing a tf.contrib.rnn.LSTMStateTuple for each cell
# init_state = multi_rnn_cell.zero_state(batch_size, tf.float32)
print('compiling the model')
outputs, final_state = tf.nn.dynamic_rnn(cell = multi_rnn_cell,
                                  inputs = x,
                                  # initial_state = init_state,
                                  dtype = tf.float32)
print('model is made')

building the model
compiling the model
model is made


In [95]:
outputs, final_state

(<tf.Tensor 'rnn/transpose:0' shape=(?, 20, 100) dtype=float32>,
 (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 100) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 100) dtype=float32>),
  LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 100) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 100) dtype=float32>),
  LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_6:0' shape=(?, 100) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_7:0' shape=(?, 100) dtype=float32>)))

### Computation of the MSE loss

In [96]:
# reshape outputs and y so that we can compute the prediction y_ and loss easily
outputs_reshaped = tf.reshape(outputs, [-1, state_size])
y_reshaped = tf.reshape(y,[-1, 2])
outputs_reshaped, y_reshaped

(<tf.Tensor 'Reshape:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(?, 2) dtype=float32>)

In [97]:
with tf.variable_scope('output'):
    V = tf.get_variable('V', [state_size, 2])
    bv = tf.get_variable('bv', [2], initializer=tf.constant_initializer(0.0))
y_hat = tf.matmul(outputs_reshaped, V) + bv
y_hat, y_reshaped

(<tf.Tensor 'add:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(?, 2) dtype=float32>)

In [98]:
total_loss = tf.reduce_mean(tf.square(tf.subtract(y_hat, y_reshaped)))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

### Training

In [125]:
X , Y = None, None
count = 0
sum_tr_losses = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        X , Y = get_batch(Xs, Ys, batch_size = batch_size)
        tr_losses, _ = sess.run([total_loss, train_step],
                                              feed_dict = {x:X, y:Y, dropout:0.2})
        count += 1
        sum_tr_losses += tr_losses
        if (i < 10) or ( i % 200 == 0 ):
            print("{} {}".format(i, sum_tr_losses / count))
            count = 0
            sum_tr_losses = 0

0 0.256749629974
1 0.0662115067244
2 0.430745214224
3 0.115134313703
4 0.0455385707319
5 0.1085588485
6 0.104458913207
7 0.060328297317
8 0.0562062039971
9 0.0344979204237
200 0.00156476524877
400 0.000450606227823
600 0.000338736219419
800 0.000265448657447
1000 0.000217243239531
1200 0.000181901800242
1400 0.000156928024371
1600 0.000137402696346
1800 0.000125454670488
